In [10]:
import pandas as pd

# --- 1. 관측소-시도 매핑 정의 및 데이터 로드 ---

# 1-1. 관측소-시도 매핑 딕셔너리 정의
sido_map = {
    '서울': '서울특별시', '부산': '부산광역시', '대구': '대구광역시',
    '광주': '광주광역시', '울산': '울산광역시', '인천': '인천광역시',
    '대전': '대전광역시',
    
    # 도 단위 및 기타 지역 매핑
    '북춘천': '강원도', '북강릉': '강원도', '울릉도': '경상북도',
    '안동': '경상북도', '포항': '경상북도', '창원': '경상남도',
    '수원': '경기도', '청주': '충청북도', '전주': '전라북도',
    '목포': '전라남도', '여수': '전라남도', '흑산도': '전라남도',
    '홍성': '충청남도', '백령도': '인천광역시', '서귀포': '제주도',
    '제주': '제주도'
}

# 1-2. 냉해 스트레스 지수 파일 로드 (이전 단계 결과 파일)
output_filename = '복숭아_냉해_스트레스_지수_2018_2024.csv'
try:
    df_features = pd.read_csv(output_filename)
except FileNotFoundError:
    print(f"오류: '{output_filename}' 파일을 찾을 수 없습니다. 냉해 지수 산출 코드를 먼저 실행하여 파일을 생성해 주세요.")
    exit()

# 1-3. 관측소명을 시도명으로 매핑
df_features['시도별'] = df_features['관측소명'].map(sido_map)

# 유효한 시도 데이터만 사용
df_features = df_features.dropna(subset=['시도별'])

# --- 2. 시도별 데이터 집계 (냉해 일수 총합 계산 및 정수형 변환) ---

# 냉해 일수 총합 계산을 위해 '냉해_스트레스_일수' 컬럼을 숫자로 변환
df_features['냉해_스트레스_일수'] = pd.to_numeric(df_features['냉해_스트레스_일수'], errors='coerce').fillna(0)

# 시도별, 연도별로 그룹화하여 냉해 일수 총합만 집계
df_sido_aggregated = df_features.groupby(['시도별', '연도']).agg(
    총합_냉해_스트레스_일수=('냉해_스트레스_일수', 'sum')
).reset_index()





# --- 3. 복숭아 생산량 데이터 로드 및 병합 ---

# 3-1. 복숭아 생산량 데이터 로드 및 정리 (Long Format으로 변환)
df_yield = pd.read_csv('복숭아 생산량18~24.csv', header=[0, 1])

# Multi-index 컬럼 정리 및 이름 변경
df_yield.columns = ['_'.join(col).strip() if col[1] != '시도별' else col[0] for col in df_yield.columns.values]
df_yield = df_yield.rename(columns={'시도별': '시도별'})

# Wide Format을 Long Format으로 변환
id_vars = ['시도별']
value_vars = [col for col in df_yield.columns if col not in id_vars]

df_yield_long = pd.melt(df_yield, id_vars=id_vars, value_vars=value_vars,
                        var_name='연도_생산량', value_name='복숭아:생산량 (톤)')

# 연도 추출 및 데이터 정리
df_yield_long['연도'] = df_yield_long['연도_생산량'].str.extract(r'(\d{4})').astype(int)
df_yield_long = df_yield_long[['시도별', '연도', '복숭아:생산량 (톤)']].copy()

# '계' 행 및 결측값 제거
df_yield_long = df_yield_long[df_yield_long['시도별'] != '계']
df_yield_long = df_yield_long.dropna(subset=['복숭아:생산량 (톤)'])


# 3-2. 시도별 집계된 특성 데이터와 생산량 데이터 병합
df_final_regression_set = df_yield_long.merge(
    df_sido_aggregated, 
    on=['시도별', '연도'], 
    how='left'
)

# 최종 데이터셋 저장
output_regression_filename = '복숭아_최종_회귀_데이터셋_1단계_정수형_총합.csv'
df_final_regression_set.to_csv(output_regression_filename, index=False)

print(f"최종 회귀 데이터셋 '{output_regression_filename}' 생성 및 저장 완료.")

최종 회귀 데이터셋 '복숭아_최종_회귀_데이터셋_1단계_정수형_총합.csv' 생성 및 저장 완료.
